In [3]:
import json
import string
import random 

import nltk
import numpy as np
from nltk.stem import WordNetLemmatizer 
import tensorflow as tf 
from tensorflow.keras import Sequential 
from tensorflow.keras.layers import Dense, Dropout
nltk.download("punkt")
nltk.download("wordnet")


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\gasse\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\gasse\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
data_file = open('intents.json',encoding="utf8").read()
data = json.loads(data_file)

data

{'intents': [{'context': '',
   'patterns': ['Hello', 'Hi there', 'Good morning', "What's up", 'Hey!'],
   'responses': ['Hey!', 'Hello', 'Hi!', 'Good morning!', "What's up"],
   'tag': 'hello'},
  {'context': [''],
   'patterns': [],
   'responses': ["Sorry, can't understand you",
    'Please give me more info',
    'Not sure I understand'],
   'tag': 'noanswer'},
  {'context': '',
   'patterns': ['What is your job',
    'What is your work',
    'What can you do?',
    'What are your features?',
    'What are you abilities'],
   'responses': ['My job is to make you feel like everything is okay.',
    'I can do whatever you asks me to do',
    'I can talk and do things for you',
    "Right now i'm in developing stage as soon i'm developed, I can do everything",
    'I work to serve you as well as possible'],
   'tag': 'job'},
  {'context': '',
   'patterns': ['What is your age', 'How old are you', 'When were you born'],
   'responses': ['I was born in 2022'],
   'tag': 'age'},
  {'cont

In [5]:
#4 Extracting data_X(features) and data_Y(Target)

words = [] #For Bow model/ vocabulary for patterns
classes = [] #For Bow  model/ vocabulary for tags
data_X = [] #For storing each pattern
data_y = [] #For storing tag corresponding to each pattern in data_X 
# Iterating over all the intents

for intent in data["intents"]:
    for pattern in intent["patterns"]:
        tokens = nltk.word_tokenize(pattern) # tokenize each pattern 
        words.extend(tokens) #and append tokens to words
        data_X.append(pattern) #appending pattern to data_X
        data_y.append(intent["tag"]) ,# appending the associated tag to each pattern 
    
    # adding the tag to the classes if it's not there already 
    if intent["tag"] not in classes:
        classes.append(intent["tag"])

# initializing lemmatizer to get stem of words
lemmatizer = WordNetLemmatizer()

# lemmatize all the words in the vocab and convert them to lowercase
# if the words don't appear in punctuation
words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in string.punctuation]
# sorting the vocab and classes in alphabetical order and taking the # set to ensure no duplicates occur
words = sorted(set(words))
classes = sorted(set(classes))

In [6]:
# 5 Text to Numbers
training = []
out_empty = [0] * len(classes)
# creating the bag of words model
for idx, doc in enumerate(data_X):
    bow = []
    text = lemmatizer.lemmatize(doc.lower())
    for word in words:
        bow.append(1) if word in text else bow.append(0)
    # mark the index of class that the current pattern is associated
    # to
    output_row = list(out_empty)
    output_row[classes.index(data_y[idx])] = 1
    # add the one hot encoded BoW and associated classes to training 
    training.append([bow, output_row])
# shuffle the data and convert it to an array
random.shuffle(training)
training = np.array(training, dtype=object)
# split the features and target labels
train_X = np.array(list(training[:, 0]))
train_Y = np.array(list(training[:, 1]))

In [7]:
#6 The Neural Network Model
model = Sequential()
model.add(Dense(128, input_shape=(len(train_X[0]),), activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(len(train_Y[0]), activation = "softmax"))
adam = tf.keras.optimizers.Adam(learning_rate=0.01, decay=1e-6)
model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=["accuracy"])
print(model.summary())
model.fit(x=train_X, y=train_Y, epochs=150, verbose=1)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               13696     
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 27)                1755      
Total params: 23,707
Trainable params: 23,707
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/150
3/3 [==============================] - 3s 10ms/step - loss: 3.3311 - accuracy: 0.0114
Epoch 2/150
3/3

3/3 [==============================] - 0s 4ms/step - loss: 0.1508 - accuracy: 0.9432
Epoch 75/150
3/3 [==============================] - 0s 4ms/step - loss: 0.2099 - accuracy: 0.8977
Epoch 76/150
3/3 [==============================] - 0s 4ms/step - loss: 0.1526 - accuracy: 0.9432
Epoch 77/150
3/3 [==============================] - 0s 2ms/step - loss: 0.2405 - accuracy: 0.8864
Epoch 78/150
3/3 [==============================] - 0s 4ms/step - loss: 0.2708 - accuracy: 0.9205
Epoch 79/150
3/3 [==============================] - 0s 4ms/step - loss: 0.2560 - accuracy: 0.9091
Epoch 80/150
3/3 [==============================] - 0s 2ms/step - loss: 0.1569 - accuracy: 0.9318
Epoch 81/150
3/3 [==============================] - 0s 4ms/step - loss: 0.1402 - accuracy: 0.9318
Epoch 82/150
3/3 [==============================] - 0s 6ms/step - loss: 0.2115 - accuracy: 0.8750
Epoch 83/150
3/3 [==============================] - 0s 4ms/step - loss: 0.1634 - accuracy: 0.9432
Epoch 84/150
3/3 [===============

In [8]:
#7 Preprocessing the Input

def clean_text(text): 
  tokens = nltk.word_tokenize(text)
  tokens = [lemmatizer.lemmatize(word.lower()) for word in tokens]
  return tokens

def bag_of_words(text, vocab): 
  tokens = clean_text(text)
  bow = [0] * len(vocab)
  for w in tokens: 
    for idx, word in enumerate(vocab):
      if word == w: 
        bow[idx] = 1
  return np.array(bow)

def pred_class(text, vocab, labels): 
  bow = bag_of_words(text, vocab)
  result = model.predict(np.array([bow]))[0] #Extracting probabilities
  thresh = 0.5
  y_pred = [[indx, res] for indx, res in enumerate(result) if res > thresh]
  y_pred.sort(key=lambda x: x[1], reverse=True) #Sorting by values of probability in decreasing order
  return_list = []
  for r in y_pred:
    return_list.append(labels[r[0]]) #Contains labels(tags) for highest probability 
  return return_list

def get_response(intents_list, intents_json): 
  if len(intents_list) == 0:
    result = "Sorry! I don't understand."
  else:
    tag = intents_list[0]
    list_of_intents = intents_json["intents"]
    for i in list_of_intents: 
      if i["tag"] == tag:
        result = random.choice(i["responses"])
        break
  return result

In [9]:
from lxml import html
import requests
from newsapi import NewsApiClient
import wikipedia
import pyowm
import webbrowser
import json
from pandas.io.json import json_normalize
import pandas as pd



In [10]:
#Creating GUI with tkinter
import tkinter
from tkinter import *
from socket import timeout
def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)
    if (msg != '') and ('--' not in msg) :
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="#442265", font=("Verdana", 12 ))
        ints = pred_class(msg, words, classes)
        res = get_response(ints, data)
        ChatLog.insert(END, "Bot: " + res + '\n\n')
          
        if 'wiki:' in msg:
            x=msg.split(":")
            for w in x :
                if w=='wiki' :
                    x.remove(w)
            msg = " ".join(x)
            print(msg)
            w=wikipedia.summary(str(msg),sentences=2)
            ChatLog.insert(END, "Bot: " + w + '\n\n')
            print(wikipedia.summary(str(msg),sentences=2))
            
            


        if'news' in msg:
            ChatLog.insert(END, "Bot: TOP NEWS : "+ '\n\n')
            newsapi = NewsApiClient(api_key='439908f4580845f696e6a19fa868cfd5')
            top_headlines = newsapi.get_top_headlines(language='en')
            scrapped_data =json.dumps(top_headlines)
            d=json.loads(scrapped_data)
            articles = d["articles"] 
            results = [arr["title"] for arr in articles] 

            for i, arr in enumerate(results, 1): 
                ChatLog.insert(END, "Bot : "+arr+ '\n\n')
                if i==5 :
                    break

            

        if 'google' in msg:
            x=msg.split()
            x.remove('google')
            msg = " ".join(x)
            new=2
            tabUrl="https://www.google.com/search?q=" 
            webbrowser.open(tabUrl+msg,new=new)
            
       
          
    
    
    
    
    if '--' in msg :
        city=''
        for i in range (len(msg)) :
            if msg[i]=='-' :
                city=msg[i+1:]
                
        ChatLog.insert(END, "You: " + city + '\n\n')    
        
        owm=pyowm.OWM('7481da4f938822e5cf6bf62834c6d5da')
        observation=owm.weather_at_place(city)
        w=observation.get_weather()
        a=w.get_wind()
        b=w.get_temperature('celsius')
        c=w.get_humidity()
        ChatLog.config(state=NORMAL) 
        ChatLog.insert(END, "Bot: in " + city +'\n\n'+"Wind : "+str(a)+ '\n\n'+"temp (celsius) : "+str(b)+'\n\n'+"humidity : "+str(c) + '\n\n')
        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)
        
    
    
    
    
    
    
        
base = Tk()
base.title("PFA")
base.geometry("400x500")
base.resizable(width=FALSE, height=FALSE)
#Create Chat window
ChatLog = Text(base, bd=0, bg="white", height="8", width="50", font="Arial",)
ChatLog.config(state=DISABLED)
#Bind scrollbar to Chat window
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set
#Create Button to send message
SendButton = Button(base, font=("Verdana",12,'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#32de97", activebackground="#3c9d9b",fg='#ffffff',
                    command= send )

#Create the box to enter message
EntryBox = Text(base, bd=0, bg="white",width="29", height="5", font="Arial")

#EntryBox.bind("<Return>", send)

#Place all components on the screen
scrollbar.place(x=376,y=6, height=386)
ChatLog.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)
SendButton.place(x=6, y=401, height=90)
base.mainloop()